In [1]:
import os
import zarr
import random
import json
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
from pathlib import Path
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch
import torchvision.transforms.functional as F
import random
import sys
from collections import defaultdict

warnings.filterwarnings("ignore")
sys.path.append("./src/")

from src.config import CFG
from src.dataloader import (
    read_zarr,
    read_info_json,
    scale_coordinates,
    create_dataset,
    create_segmentation_map,
    EziiDataset,
    drop_padding,
)
from src.network import UNet_2D, aug
from src.utils import save_images
from src.metric import score, create_cls_pos, create_cls_pos_sikii, create_df

sample_submission = pd.read_csv("../../inputs/sample_submission.csv")

In [ ]:
train_dataset = EziiDataset(
    exp_names=CFG.train_exp_names,
    base_dir="../../inputs/train",
    particles_name=CFG.particles_name,
    resolution=CFG.resolution,
    zarr_type=CFG.train_zarr_types,
    train=True,
)

valid_dataset = EziiDataset(
    exp_names=CFG.valid_exp_names,
    # exp_names=CFG.train_exp_names,
    base_dir="../../inputs/train",
    particles_name=CFG.particles_name,
    resolution=CFG.resolution,
    zarr_type=CFG.valid_zarr_types,
    train=True,
)

from tqdm import tqdm

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False)

for row in tqdm(valid_loader):
    normalized_tomogram = row["normalized_tomogram"]
    break

In [3]:
class PadToSize(nn.Module):
    def __init__(self, resolution):
        super().__init__()
        if resolution == "0":
            self.size = 640
        elif resolution == "1":
            self.size = 320
        elif resolution == "2":
            self.size = 160

    def forward(self, x):
        return F.pad(x, (0, 0, self.size - x.shape[-1], self.size - x.shape[-2]))

In [4]:
model = UNet_2D().to("cuda")
model.eval()
model.load_state_dict(torch.load("./best_model.pth"))


optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(
    weight=torch.tensor([0.5, 32, 32, 32, 32, 32, 32]).to("cuda")
)
# criterion = DiceLoss()

best_model = None
best_loss = np.inf
batch_size = 4

valid_loss = []
valid_pred_tomogram = defaultdict(list)
valid_gt_tomogram = defaultdict(list)
model.eval()
tq = tqdm(range(len(valid_loader) * normalized_tomogram.shape[0]))
for data in valid_loader:
    exp_name = data["exp_name"][0]
    tomogram = data["normalized_tomogram"].to("cuda")
    segmentation_map = data["segmentation_map"].to("cuda").long()

    for i in range(tomogram.shape[1]):
        input_ = tomogram[:, i].unsqueeze(0)
        gt = segmentation_map[:, i]

        input_ = PadToSize(CFG.resolution)(input_)
        gt = PadToSize(CFG.resolution)(gt)
        output = model(input_)
        output = nn.functional.softmax(output, dim=1)
        loss = criterion(output, gt)

        valid_loss.append(loss.item())
        tq.set_description(f"Loss: {np.mean(valid_loss)}")
        tq.update(1)

        output = drop_padding(output, CFG.resolution)

        valid_pred_tomogram[exp_name].append(output.cpu().detach().numpy())
        valid_gt_tomogram[exp_name].append(gt.cpu().detach().numpy())
tq.close()

Loss: 1.3900193069940028: : 368it [00:17, 21.15it/s]


In [5]:
def create_gt_df(base_dir, exp_names):
    result_df = None
    particle_names = CFG.particles_name

    for exp_name in exp_names:
        for particle in particle_names:
            np_corrds = read_info_json(
                base_dir=base_dir, exp_name=exp_name, particle_name=particle
            )  # (n, 3)
            # 各行にexp_nameとparticle_name追加
            particle_df = pd.DataFrame(np_corrds, columns=["z", "y", "x"])
            particle_df["experiment"] = exp_name
            particle_df["particle_type"] = particle

            if result_df is None:
                result_df = particle_df
            else:
                result_df = pd.concat([result_df, particle_df], axis=0).reset_index(
                    drop=True
                )

    result_df = result_df.reset_index()  # index	experiment	particle_type	x	y	z
    result_df = result_df[["index", "experiment", "particle_type", "x", "y", "z"]]

    return result_df

In [6]:
gt_df = create_gt_df("../../inputs/train/overlay/ExperimentRuns/", CFG.valid_exp_names)
gt_df = gt_df[gt_df["particle_type"] != "beta-amylase"].reset_index(drop=True)

In [7]:
# pred_df = pd.read_csv("../../inputs/train_submission.csv")


def calc_score(initial_sikii):
    all_pred_df = None

    for exp_name in CFG.valid_exp_names:
        pred_tomogram = valid_pred_tomogram[exp_name]
        pred_tomogram = np.array(pred_tomogram)  # (92, 1, 7, 315, 315)
        pred_tomogram = pred_tomogram.squeeze(1)  # (92, 7, 315, 315)

        pred_cls_pos, pred_Ascale_pos = create_cls_pos_sikii(
            pred_tomogram, sikii_dict=initial_sikii
        )
        pred_df = create_df(pred_Ascale_pos, exp_name)
        # pred_df = create_df(pred_cls_pos, exp_name)

        if all_pred_df is None:
            all_pred_df = pred_df
        else:
            all_pred_df = pd.concat([all_pred_df, pred_df], axis=0).reset_index(
                drop=True
            )

    pred_df = all_pred_df[all_pred_df["particle_type"] != "beta-amylase"]
    pred_df = pred_df.drop_duplicates(subset=["x", "y", "z"], keep="first").reset_index(
        drop=True
    )

    pred_df = pred_df.reset_index()

    score_ = score(
        pred_df, gt_df, row_id_column_name="index", distance_multiplier=1, beta=4
    )

    return score_


def calc_score_by_exp(initial_sikii):
    exp_scores = {}

    for exp_name in CFG.valid_exp_names:
        gt_df = create_gt_df("../../inputs/train/overlay/ExperimentRuns/", [exp_name])

        pred_tomogram = valid_pred_tomogram[exp_name]
        pred_tomogram = np.array(pred_tomogram)  # (92, 1, 7, 315, 315)
        pred_tomogram = pred_tomogram.squeeze(1)  # (92, 7, 315, 315)

        pred_cls_pos, pred_Ascale_pos = create_cls_pos_sikii(
            pred_tomogram, sikii_dict=initial_sikii
        )
        pred_df = create_df(pred_Ascale_pos, exp_name)

        pred_df = pred_df[pred_df["particle_type"] != "beta-amylase"]
        pred_df = pred_df.drop_duplicates(
            subset=["x", "y", "z"], keep="first"
        ).reset_index(drop=True)

        pred_df = pred_df.reset_index()

        score_ = score(
            pred_df, gt_df, row_id_column_name="index", distance_multiplier=1, beta=4
        )

        exp_scores[exp_name] = score_

    return exp_scores

In [8]:
constant = 0.35

initial_sikii = {
    "apo-ferritin": constant,
    "beta-amylase": constant,
    "beta-galactosidase": constant,
    "ribosome": constant,
    "thyroglobulin": constant,
    "virus-like-particle": constant,
}

score_ = calc_score(initial_sikii)
score_

0.40905056505945697

In [9]:
constant = 0.35

initial_sikii = {
    "apo-ferritin": constant,
    "beta-amylase": constant,
    "beta-galactosidase": constant,
    "ribosome": constant,
    "thyroglobulin": constant,
    "virus-like-particle": constant,
}

score_ = calc_score_by_exp(initial_sikii)
score_

{'TS_86_3': 0.4891645161984996, 'TS_6_6': 0.32778489591101295}

In [10]:
best_sikii = 0
best_score = -np.inf

for sikii in np.linspace(0.3, 0.7, 100):
    initial_sikii = {
        "apo-ferritin": sikii,
        "beta-amylase": sikii,
        "beta-galactosidase": sikii,
        "ribosome": sikii,
        "thyroglobulin": sikii,
        "virus-like-particle": sikii,
    }
    score_ = calc_score(initial_sikii)
    if score_ > best_score:
        best_score = score_
        best_sikii = sikii
    print(sikii, score_)

0.25 0.36800004366629324
0.25202020202020203 0.37121187025437813
0.25404040404040407 0.37147745064740506
0.25606060606060604 0.372337397464804
0.2580808080808081 0.37440163157323697
0.2601010101010101 0.3748863455423777
0.26212121212121214 0.3757872057637299
0.2641414141414141 0.3765742670927833
0.26616161616161615 0.3810862487128103
0.2681818181818182 0.38125061856186443
0.2702020202020202 0.3803468781414561
0.2722222222222222 0.38125061856186443
0.27424242424242423 0.38319018423286166
0.27626262626262627 0.38319018423286166
0.2782828282828283 0.3843242854559256
0.2803030303030303 0.38871518329401195
0.2823232323232323 0.38168590658685514
0.28434343434343434 0.3818364510556787
0.2863636363636364 0.3827369846215912
0.2883838383838384 0.38093820851302207
0.2904040404040404 0.3837330744657828
0.2924242424242424 0.38225237994322253
0.29444444444444445 0.38328759351582936
0.2964646464646465 0.38631377037639264
0.29848484848484846 0.38809948466210686
0.3005050505050505 0.38756133010165306
0

In [11]:
best_sikii, best_score

(0.3691919191919192, 0.4212366475482799)

# Train-Dataset

In [12]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(
    weight=torch.tensor([0.5, 32, 32, 32, 32, 32, 32]).to("cuda")
)
# criterion = DiceLoss()

best_model = None
best_loss = np.inf
batch_size = 4

train_loss = []
valid_pred_tomogram = defaultdict(list)
valid_gt_tomogram = defaultdict(list)
model.eval()
tq = tqdm(range(len(train_loader) * normalized_tomogram.shape[0]))
for data in train_loader:
    exp_name = data["exp_name"][0]
    tomogram = data["normalized_tomogram"].to("cuda")
    segmentation_map = data["segmentation_map"].to("cuda").long()

    for i in range(tomogram.shape[1]):
        input_ = tomogram[:, i].unsqueeze(0)
        gt = segmentation_map[:, i]

        input_ = PadToSize(CFG.resolution)(input_)
        gt = PadToSize(CFG.resolution)(gt)
        output = model(input_)
        output = nn.functional.softmax(output, dim=1)
        loss = criterion(output, gt)

        train_loss.append(loss.item())
        tq.set_description(f"Loss: {np.mean(train_loss)}")
        tq.update(1)

        output = drop_padding(output, CFG.resolution)

        valid_pred_tomogram[exp_name].append(output.cpu().detach().numpy())
        valid_gt_tomogram[exp_name].append(gt.cpu().detach().numpy())
tq.close()

Loss: 1.35843947565426: : 3680it [04:45, 12.91it/s]                      


In [13]:
def calc_score(initial_sikii):
    all_pred_df = None

    for exp_name in CFG.train_exp_names:
        pred_tomogram = valid_pred_tomogram[exp_name]
        pred_tomogram = np.array(pred_tomogram)  # (92, 1, 7, 315, 315)
        pred_tomogram = pred_tomogram.squeeze(1)  # (92, 7, 315, 315)

        pred_cls_pos, pred_Ascale_pos = create_cls_pos_sikii(
            pred_tomogram, sikii_dict=initial_sikii
        )
        pred_df = create_df(pred_Ascale_pos, exp_name)
        # pred_df = create_df(pred_cls_pos, exp_name)

        if all_pred_df is None:
            all_pred_df = pred_df
        else:
            all_pred_df = pd.concat([all_pred_df, pred_df], axis=0).reset_index(
                drop=True
            )

    pred_df = all_pred_df[all_pred_df["particle_type"] != "beta-amylase"]
    pred_df = pred_df.drop_duplicates(subset=["x", "y", "z"], keep="first").reset_index(
        drop=True
    )

    pred_df = pred_df.reset_index()

    score_ = score(
        pred_df, gt_df, row_id_column_name="index", distance_multiplier=1, beta=4
    )

    return score_

In [14]:
gt_df = create_gt_df("../../inputs/train/overlay/ExperimentRuns/", CFG.train_exp_names)
gt_df = gt_df[gt_df["particle_type"] != "beta-amylase"].reset_index(drop=True)

In [ ]:
best_sikii = 0
best_score = -np.inf

for sikii in np.linspace(0.25, 0.45, 100):
    initial_sikii = {
        "apo-ferritin": sikii,
        "beta-amylase": sikii,
        "beta-galactosidase": sikii,
        "ribosome": sikii,
        "thyroglobulin": sikii,
        "virus-like-particle": sikii,
    }
    score_ = calc_score(initial_sikii)
    if score_ > best_score:
        best_score = score_
        best_sikii = sikii
    print(sikii, score_)

0.25 0.2158016280706467
0.25202020202020203 0.21571803316555968
0.25404040404040407 0.2164215309217528
0.25606060606060604 0.21706368654184563
0.2580808080808081 0.2178370344852624
0.2601010101010101 0.21818796410467947
0.26212121212121214 0.21832081553992375
0.2641414141414141 0.21837685013150118
0.26616161616161615 0.21852894902050662
0.2681818181818182 0.21859694292454002
0.2702020202020202 0.21818692123634978
0.2722222222222222 0.21831692195465724
0.27424242424242423 0.21886794983117955
